In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

In [2]:
# Read School Data File and store into Pandas DataFrame
school_data = pd.read_csv(school_data_to_load)

In [3]:
# Read School Data File and store into Pandas DataFrame
student_data = pd.read_csv(student_data_to_load)

In [4]:
# Combine the data into a single dataset.  
school_df = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_df.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [5]:
#Calculate total number of schools in the district
total_schools = school_df["school_name"].nunique()

#Calculate the total number of students in the district
total_students = school_df["Student ID"].nunique()

#Calculate the total budget
school_budgets = school_df["budget"].unique()
total_budget = school_budgets.sum()

#Calculate the average math score
avg_math_scores = school_df["math_score"].mean()

#Calculate the average reading score
avg_reading_scores = school_df["reading_score"].mean()

#Calculate the percentage of students with a passing math score (70 or greater)
passing_math_df = school_df.loc[school_df["math_score"] >= 70]

students_passed_math = passing_math_df["Student ID"].nunique()

#Calculate the percentage of students passing math
percent_passing_math = students_passed_math / total_students

#Calculate the percentage of students with a passing reading score (70 or greater)
passing_reading_df = school_df.loc[school_df["reading_score"] >= 70]

students_passed_reading = passing_reading_df["Student ID"].nunique()

#Calculate the percentage of students passing math
percent_passing_reading = students_passed_reading / total_students

#Calculate the percentage of students with a passing reading AND math score (70 or greater)
passing_df = school_df.loc[(school_df["reading_score"] >= 70) & (school_df["math_score"] >= 70)]

students_passed_both = passing_df["Student ID"].nunique()
percent_passing_both = students_passed_both / total_students

In [6]:
#Create a summary df to store district results
district_df = pd.DataFrame({"Total Schools": [total_schools],
                            "Total Students": total_students,
                            "Total Budget": total_budget,
                            "Average Math Score": avg_math_scores,
                            "Average Reading Score": avg_reading_scores,
                            "% Passing Math": percent_passing_math,
                            "% Passing Reading": percent_passing_reading,
                            "% Overall Passing": percent_passing_both})

#Formatting the fields in the district df
district_df["Total Budget"] = district_df["Total Budget"].astype(float).map("${:,.2f}".format)
district_df["Average Math Score"] = district_df["Average Math Score"].astype(float).map("{:,.2f}".format)
district_df["Average Reading Score"] = district_df["Average Reading Score"].astype(float).map("{:,.2f}".format)
district_df["% Passing Math"] = district_df["% Passing Math"].astype(float).map("{:.2%}".format)
district_df["% Passing Reading"] = district_df["% Passing Reading"].astype(float).map("{:.2%}".format)
district_df["% Overall Passing"] = district_df["% Overall Passing"].astype(float).map("{:.2%}".format)

district_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,39170,"$24,649,428.00",78.99,81.88,74.98%,85.81%,65.17%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [30]:
# Define the school type and tie it to an index of the school names
school_type = school_data.set_index(["school_name"])["type"]

# Find the number of students per school
students_per_school = school_df["school_name"].value_counts()

# Find the total budget per school
budget_per_school = school_data.groupby("school_name").sum()["budget"]

# Use above values to find total budget per student by school
budget_per_student = budget_per_school / students_per_school

# Find the average math score by school and identify the percentage of students with a passing grade (>= 70)
avg_math_by_school = school_df.groupby("school_name").mean()["math_score"]

math_pass_perc = (passing_math_df.groupby("school_name").count()["math_score"] / students_per_school) * 100

# Find the average reading score by school and identify the percentage of students with a passing grade (>= 70)
avg_reading_by_school = school_df.groupby("school_name").mean()["reading_score"]

reading_pass_perc = (passing_reading_df.groupby("school_name").count()["reading_score"] / students_per_school) * 100

# Calculate the percentage of students per school passing both math & reading
students_pass_both = passing_math_df.loc[passing_math_df["reading_score"] >= 70]

total_kids_math = students_pass_both.groupby("school_name").count()["math_score"] / students_per_school
total_kids_reading = students_pass_both.groupby("school_name").count()["reading_score"] / students_per_school

both_pass_perc = ((total_kids_math + total_kids_reading) / 2) * 100

In [32]:
#Create a summary df to store school specific results
school_summ_df = pd.DataFrame({
    "School Type": school_type,
    "Total Students": students_per_school,
    "Total School Budget": budget_per_school,
    "Per Student Budget": budget_per_student,
    "Average Math Score": avg_math_by_school,
    "Average Reading Score": avg_reading_by_school,
    "% Passing Math": math_pass_perc,
    "% Passing Reading": reading_pass_perc,
    "% Overall Passing": both_pass_perc})

school_summ_df['Total Students'] = school_summ_df['Total Students'].astype(float).map("{:,.0f}".format)
school_summ_df['Total School Budget'] = school_summ_df['Total School Budget'].astype(float).map("${:,.2f}".format)
school_summ_df['Per Student Budget'] = school_summ_df['Per Student Budget'].astype(float).map("${:,.2f}".format)
school_summ_df["Average Math Score"] = school_summ_df["Average Math Score"].astype(float)
school_summ_df["Average Reading Score"] = school_summ_df["Average Reading Score"].astype(float)
school_summ_df['% Passing Math'] = school_summ_df['% Passing Math'].astype(float)
school_summ_df['% Passing Reading'] = school_summ_df['% Passing Reading'].astype(float)
school_summ_df['% Overall Passing'] = school_summ_df['% Overall Passing'].astype(float)

school_summ_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [9]:
top_school_summary = school_summ_df.sort_values(["% Overall Passing"], ascending = False)
top_school_summary.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.06,83.98,94.13%,97.04%,91.33%
Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.42,83.85,93.27%,97.31%,90.95%
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.35,83.82,93.39%,97.14%,90.60%
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.27,83.99,93.87%,96.54%,90.58%
Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,94.59%,95.95%,90.54%


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [10]:
bottom_school_summary = school_summ_df.sort_values(["% Overall Passing"], ascending = True)
bottom_school_summary.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.84,80.74,66.37%,80.22%,52.99%
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.71,81.16,65.99%,80.74%,53.20%
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.63,81.18,65.68%,81.32%,53.51%
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.29,80.93,66.75%,80.86%,53.53%
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.07,80.97,66.06%,81.22%,53.54%


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [11]:
# Retrieve students by grade and find the average math scores by those grades at each school
ninth_grade = school_df[school_df["grade"] == "9th"]
tenth_grade = school_df[school_df["grade"] == "10th"]
eleventh_grade = school_df[school_df["grade"] == "11th"]
twelfth_grade = school_df[school_df["grade"] == "12th"]

math_ninth = ninth_grade.groupby("school_name")["math_score"].mean()
math_tenth = tenth_grade.groupby("school_name")["math_score"].mean()
math_eleventh = eleventh_grade.groupby("school_name")["math_score"].mean()
math_twelfth = twelfth_grade.groupby("school_name")["math_score"].mean()

In [12]:
# Create a summary df to display math scores by grade at each school
math_score_by_grade = pd.DataFrame({
    "9th Grade": math_ninth,
    "10th Grade": math_tenth,
    "11th Grade": math_eleventh,
    "12th Grade": math_twelfth
})

# Clean up formatting
math_score_by_grade["9th Grade"] = math_score_by_grade["9th Grade"].astype(float).map("{:,.2f}".format)
math_score_by_grade["10th Grade"] = math_score_by_grade["10th Grade"].astype(float).map("{:,.2f}".format)
math_score_by_grade["11th Grade"] = math_score_by_grade["11th Grade"].astype(float).map("{:,.2f}".format)
math_score_by_grade["12th Grade"] = math_score_by_grade["12th Grade"].astype(float).map("{:,.2f}".format)


math_score_by_grade

,9th Grade,10th Grade,11th Grade,12th Grade
school_name,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [13]:
# Using students by grade (previously defined), find the average reading scores by those grades at each school
reading_ninth = ninth_grade.groupby("school_name")["reading_score"].mean()
reading_tenth = tenth_grade.groupby("school_name")["reading_score"].mean()
reading_eleventh = eleventh_grade.groupby("school_name")["reading_score"].mean()
reading_twelfth = twelfth_grade.groupby("school_name")["reading_score"].mean()

In [14]:
# Create a summary df to display reading scores by grade at each school
reading_score_by_grade = pd.DataFrame({
    "9th Grade": reading_ninth,
    "10th Grade": reading_tenth,
    "11th Grade": reading_eleventh,
    "12th Grade": reading_twelfth
})

# Clean up formatting
reading_score_by_grade["9th Grade"] = reading_score_by_grade["9th Grade"].astype(float).map("{:,.2f}".format)
reading_score_by_grade["10th Grade"] = reading_score_by_grade["10th Grade"].astype(float).map("{:,.2f}".format)
reading_score_by_grade["11th Grade"] = reading_score_by_grade["11th Grade"].astype(float).map("{:,.2f}".format)
reading_score_by_grade["12th Grade"] = reading_score_by_grade["12th Grade"].astype(float).map("{:,.2f}".format)


reading_score_by_grade

,9th Grade,10th Grade,11th Grade,12th Grade
school_name,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [34]:
# Create bins
school_spending_bins = [0, 585, 630, 645, 680]
spending_bin_labels = ["< $585", "$585-630", "$630-645", "$645-680"]

# Create spending ranges column
school_summ_df["Spending Ranges (Per Student)"] = pd.cut(budget_per_student, school_spending_bins, 
                                                           labels=spending_bin_labels, 
                                                           include_lowest=True)

# Define the averages by school spending
spending_avg_math = school_summ_df.groupby("Spending Ranges (Per Student)")["Average Math Score"].mean()
spending_avg_reading = school_summ_df.groupby("Spending Ranges (Per Student)")["Average Reading Score"].mean()
spending_pass_math = school_summ_df.groupby("Spending Ranges (Per Student)")["% Passing Math"].mean()
spending_pass_reading = school_summ_df.groupby("Spending Ranges (Per Student)")["% Passing Reading"].mean()
spending_pass_both = school_summ_df.groupby("Spending Ranges (Per Student)")["% Overall Passing"].mean()

In [35]:
# Create scores by spending df
scores_by_spending = pd.DataFrame({
    "Average Math Score": spending_avg_math,
    "Average Reading Score": spending_avg_reading,
    "% Passing Math": spending_pass_math,
    "% Passing Reading": spending_pass_reading,
    "% Overall Passing": spending_pass_both
})

scores_by_spending

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
< $585,83.455399,83.933814,93.460096,96.610877,90.369459
$585-630,81.899826,83.155286,87.133538,92.718205,81.418596
$630-645,78.518855,81.624473,73.484209,84.391793,62.857656
$645-680,76.997210,81.027843,66.164813,81.133951,53.526855


## Scores by School Size

* Perform the same operations as above, based on school size.

In [39]:
# Create bins
school_size_bins = [0, 999, 1999, 5000]
size_bin_labels = ["Small (< 1000)", "Medium (1000 - 2000)", "Large (2000 - 5000)"]

# Create size ranges column
school_summ_df["School Size"] = pd.cut(students_per_school, school_size_bins, 
                                                           labels=size_bin_labels, 
                                                           include_lowest=True)

# Define average scores grouped by school size
size_avg_math = school_summ_df.groupby("School Size")["Average Math Score"].mean()
size_avg_reading = school_summ_df.groupby("School Size")["Average Reading Score"].mean()
size_pass_math = school_summ_df.groupby("School Size")["% Passing Math"].mean()
size_pass_reading = school_summ_df.groupby("School Size")["% Passing Reading"].mean()
size_pass_both = school_summ_df.groupby("School Size")["% Overall Passing"].mean()


In [40]:
# Create scores by school size df
scores_by_size = pd.DataFrame({
    "Average Math Score": size_avg_math,
    "Average Reading Score": size_avg_reading,
    "% Passing Math": size_pass_math,
    "% Passing Reading": size_pass_reading,
    "% Overall Passing": size_pass_both
})

scores_by_size

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (< 1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium (1000 - 2000),83.374684,83.864438,93.599695,96.790680,90.621535
Large (2000 - 5000),77.746417,81.344493,69.963361,82.766634,58.286003


## Scores by School Type

* Perform the same operations as above, based on school type

In [41]:
# Define average scores grouped by school type
type_avg_math = school_summ_df.groupby("School Type")["Average Math Score"].mean()
type_avg_reading = school_summ_df.groupby("School Type")["Average Reading Score"].mean()
type_pass_math = school_summ_df.groupby("School Type")["% Passing Math"].mean()
type_pass_reading = school_summ_df.groupby("School Type")["% Passing Reading"].mean()
type_pass_both = school_summ_df.groupby("School Type")["% Overall Passing"].mean()

In [42]:
# Create scores by school type df
scores_by_type = pd.DataFrame({
    "Average Math Score": type_avg_math,
    "Average Reading Score": type_avg_reading,
    "% Passing Math": type_pass_math,
    "% Passing Reading": type_pass_reading,
    "% Overall Passing": type_pass_both
})

scores_by_type

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208
